In [2]:
import pandas as pd
import numpy as np

In [3]:
company = pd.read_excel("companydataset.xlsx")
company.head()

,Unnamed: 0,com_name,industry,job_title,job_description,prerequisites,Unnamed: 6,location,is_startup,company_size,student_rank,salary
0,NaN,Tata Consultancy Services,Technology Services & Consulting,"Software Engineer,Front-End Developer ,Databas...",Join our team to develop scalable software sol...,"cpp,dsa,javascript,sql",NaN,Mumbai,0,large,all,600000
1,NaN,Google,Technology,"ml Engineer,Software Developer,MERN stack deve...",Build cutting-edge web applications and machin...,"webdev,python,ml,datascience,MERN",NaN,"Delhi,Banglore,Hyderabad",0,large,advanced,3000000
2,NaN,Infosys,Technology Services & Consulting,"Java Developer,Mobile Application Developer,Da...",Develop enterprise-level applications with Jav...,"java,c,flutter,javascript,sql",NaN,"Bangalore,Hyderabad,Pune",0,large,all,900000
3,NaN,Microsoft,Technology,"ml Engineer,Backend Developer,Full stack devel...",Design and implement intelligent systems with ...,"ml,python,MERN,flask,django",NaN,"Banglore,Hyderabad,Noida",0,large,advanced,3000000
4,NaN,Amazon,"E-commerce, Cloud Computing, AI","Software Development Engineer,Data Analyst,ml ...",Pioneer machine learning models that drive inn...,"ml,python,datascience,cpp,dsa",NaN,"Bangalore,Hyderabad",0,large,advanced,3000000


In [4]:
company.drop(['Unnamed: 0','Unnamed: 6'],axis=1,inplace=True)


In [5]:
students = pd.read_excel("studentsdataset.xlsx")


In [6]:
company['prerequisites'] = company['prerequisites'].str.replace(',',' ')
company['prerequisites'] = company['prerequisites'].str.lower()
company['job_title'] = company['job_title'].str.lower()
company['job_description'] = company['job_description'].str.lower()
company['location'] = company['location'].str.lower()



In [7]:
students['skills'] = students['skills'].str.replace(',',' ')

In [8]:
# from sklearn.preprocessing import LabelEncoder
# label1 = LabelEncoder()
# label2 = LabelEncoder()
# label3 = LabelEncoder()
# students['preferred_job_type'] = label1.fit_transform(students['preferred_job_type'])
# students['preferred_job_size'] = label2.fit_transform(students['preferred_job_size'])
# students['Category'] = label3.fit_transform(students['Category'])


In [9]:
students.head()

,student_id,student_name,year,CGPA,skills,preferred_job_type,preferred_job_location,preferred_job_size,preferred_role,Category,expected_salary_pa,expected_salary_pm
0,232,Shubh Gupta,4,9.69,cpp python dsa ml nlp dl,internship,Mumbai,midsize,ml intern,advanced,0,68000.0
1,197,Prakarsh Singh,4,9.42,webdev python sql dsa,fulltime,Delhi,Startup,front end developer,advanced,950000,0.0
2,201,Devyanshi Saini,4,9.47,java c flutter,both,Banglore,Large,app developer,intermediate,650000,24000.0
3,216,Purvi Wasan,4,9.21,ml python,fulltime,Banglore,Large,ml engineer,intermediate,1000000,0.0
4,214,Darshan Raj,4,8.54,generative ai python ml data science,fulltime,Gurugram,midsize,python developer,rookie,250000,0.0


In [10]:
company.head()

,com_name,industry,job_title,job_description,prerequisites,location,is_startup,company_size,student_rank,salary
0,Tata Consultancy Services,Technology Services & Consulting,"software engineer,front-end developer ,databas...",join our team to develop scalable software sol...,cpp dsa javascript sql,mumbai,0,large,all,600000
1,Google,Technology,"ml engineer,software developer,mern stack deve...",build cutting-edge web applications and machin...,webdev python ml datascience mern,"delhi,banglore,hyderabad",0,large,advanced,3000000
2,Infosys,Technology Services & Consulting,"java developer,mobile application developer,da...",develop enterprise-level applications with jav...,java c flutter javascript sql,"bangalore,hyderabad,pune",0,large,all,900000
3,Microsoft,Technology,"ml engineer,backend developer,full stack devel...",design and implement intelligent systems with ...,ml python mern flask django,"banglore,hyderabad,noida",0,large,advanced,3000000
4,Amazon,"E-commerce, Cloud Computing, AI","software development engineer,data analyst,ml ...",pioneer machine learning models that drive inn...,ml python datascience cpp dsa,"bangalore,hyderabad",0,large,advanced,3000000


 ### Skills-prerequisites

In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [12]:


lemma = WordNetLemmatizer()
def lemmatize_skills(skills_str):
    skills = [skill.strip() for skill in skills_str.split(' ')]
    lemmatized_skills = []
    for skill in skills:
        tokens = word_tokenize(skill.lower())
        lemmas = [lemma.lemmatize(token) for token in tokens] 
        lemmatized_skills.append(" ".join(lemmas))
    return " ".join(lemmatized_skills) 

students['lemmatized_skills'] = students['skills'].apply(lemmatize_skills)
company['lemmatized_skills'] = company['prerequisites'].apply(lemmatize_skills)

tfidf = TfidfVectorizer()
student_skill_vectors = tfidf.fit_transform(students['lemmatized_skills'])
company_skill_vectors = tfidf.transform(company['lemmatized_skills'])

similarity_score = cosine_similarity(student_skill_vectors,company_skill_vectors)


def recommend_companies_skills_prerequisites(student_name_input):
    # student_name_input = input("Enter student name: ")
    try:
        student_index = students[students['student_name'].str.lower() == student_name_input.lower()].index[0]
    except IndexError:
        print(f"Student with name '{student_name_input}' not found.")
        exit()
        
    company_indices = similarity_score[student_index].argsort()[::-1][:19]
    recommended_companies_skill = []
    weight_skill = 0.5
    for company_index in company_indices:
        # print(f"  - {company['com_name'][company_index]}")
        company_name = company['com_name'][company_index]
        similarity = similarity_score[student_index, company_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_companies_skill.append((company_name,weighted_similarity))
    recommended_companies_skill.sort(key=lambda x : x[1],reverse=True)    
    return recommended_companies_skill    



In [13]:

def recommend_students_skills(company_name_input):
    try:
        company_index = company[company['com_name'].str.lower() == company_name_input.lower()].index[0]
    except IndexError:
        print(f"Company with name '{company_name_input}' not found.")
        return 

    # print(f"Recommendations for {company['com_name'][company_index]}:")
    student_indices = similarity_score[:, company_index].argsort()[::-1][:19]
    recommended_students_skill = []
    weight_skill = 0.5
    for student_index in student_indices:
        # print(f"  - {students['student_name'][student_index]}")
        student_name = students['student_name'][student_index]
        similarity = similarity_score[student_index, company_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_students_skill.append((student_name,weighted_similarity))
    recommended_students_skill.sort(key=lambda x : x[1],reverse=True)    
    return recommended_students_skill    
        
       

### preferred role - job title

In [14]:
lemma = WordNetLemmatizer()
def lemmatize_skills(skills_str):
    skills = [skill.strip() for skill in skills_str.split(' ')]
    lemmatized_skills = []
    for skill in skills:
        tokens = word_tokenize(skill.lower())
        lemmas = [lemma.lemmatize(token) for token in tokens] 
        lemmatized_skills.append(" ".join(lemmas))
    return " ".join(lemmatized_skills) 

students['lemmatized_preferred_role'] = students['preferred_role'].apply(lemmatize_skills)
company['lemmatized_job_title'] = (company['job_title'] + " " + company['job_description']).apply(lemmatize_skills)

tfid = TfidfVectorizer()
students_preferred_role_vectors = tfid.fit_transform(students['lemmatized_preferred_role'])
company_job_title_vectors = tfid.transform(company['lemmatized_job_title'])

similarity_score_jobtitle_role = cosine_similarity(students_preferred_role_vectors,company_job_title_vectors)


In [15]:
def recommend_companies_preferredrole_jobtitle(student_name_input):
    # student_name_input = input("Enter student name: ")
    try:
        student_index = students[students['student_name'].str.lower() == student_name_input.lower()].index[0]
    except IndexError:
        print(f"Student with name '{student_name_input}' not found.")
        exit()

    # student_name = students['student_name'][student_index]
    # print(f"Recommendations for {student_name}:")
    company_indices = similarity_score_jobtitle_role[student_index].argsort()[::-1][:19]
    recommended_companies_jobtitle_role = []
    weight_skill = 0.6
    for company_index in company_indices:
        # print(f"  - {company['com_name'][company_index]}")
        company_name = company['com_name'][company_index]
        similarity = similarity_score_jobtitle_role[student_index, company_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_companies_jobtitle_role.append((company_name,weighted_similarity))
    recommended_companies_jobtitle_role.sort(key=lambda x : x[1],reverse=True)    
    return recommended_companies_jobtitle_role    


In [16]:
def recommend_students_preferredrole_jobtitle(company_name_input):
    try:
        company_index = company[company['com_name'].str.lower() == company_name_input.lower()].index[0]
    except IndexError:
        print(f"Company with name '{company_name_input}' not found.")
        return 

    # print(f"Recommendations for {company['com_name'][company_index]}:")
    student_indices = similarity_score_jobtitle_role[:, company_index].argsort()[::-1][:10]
    recommended_students_jobtitle_role = []
    weight_skill = 0.6
    for student_index in student_indices:
        # print(f"  - {students['student_name'][student_index]}")
        student_name = students['student_name'][student_index]
        similarity = similarity_score_jobtitle_role[student_index, company_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_students_jobtitle_role.append((student_name,weighted_similarity))
    recommended_students_jobtitle_role.sort(key=lambda x : x[1],reverse=True)    
    return recommended_students_jobtitle_role
        
  

### location-location

In [17]:
lemma = WordNetLemmatizer()
def lemmatize_skills(skills_str):
    skills = [skill.strip() for skill in skills_str.split(' ')]
    lemmatized_skills = []
    for skill in skills:
        tokens = word_tokenize(skill.lower())
        lemmas = [lemma.lemmatize(token) for token in tokens] 
        lemmatized_skills.append(" ".join(lemmas))
    return " ".join(lemmatized_skills)

students['lemmatized_location'] = students['preferred_job_location'].apply(lemmatize_skills)
company['lemmatized_location'] = company['location'].apply(lemmatize_skills)

tfi = TfidfVectorizer()
students_location_vectors = tfi.fit_transform(students['lemmatized_location'])
company_location_vectors = tfi.transform(company['lemmatized_location'])

similarity_score_location = cosine_similarity(students_location_vectors,company_location_vectors)

In [18]:
def recommend_companies_location_location(student_name_input):
    
    try:
        student_index = students[students['student_name'].str.lower() == student_name_input.lower()].index[0]
    except IndexError:
        print(f"Student with name '{student_name_input}' not found.")
        exit()

    # student_name = students['student_name'][student_index]
    # print(f"Recommendations for {student_name}:")
    company_indices = similarity_score_location[student_index].argsort()[::-1][:19]
    recommended_companies_location = []
    weight_skill = 0.2
    for company_index in company_indices:
        # print(f"  - {company['com_name'][company_index]}")
        company_name = company['com_name'][company_index]
        similarity = similarity_score_location[student_index, company_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_companies_location.append((company_name,weighted_similarity))
    recommended_companies_location.sort(key=lambda x : x[1],reverse=True)    
    return recommended_companies_location 


In [19]:
def recommend_students_location(company_name_input):
    try:
        company_index = company[company['com_name'].str.lower() == company_name_input.lower()].index[0]
    except IndexError:
        print(f"Company with name '{company_name_input}' not found.")
        return 

    # print(f"Recommendations for {company['com_name'][company_index]}:")
    student_indices = similarity_score_location[:, company_index].argsort()[::-1][:10]
    recommended_students_location = []
    weight_skill = 0.2
    for student_index in student_indices:
        # print(f"  - {company['com_name'][company_index]}")
        student_name = students['student_name'][student_index]
        similarity = similarity_score_location[student_index, company_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_students_location.append((student_name,weighted_similarity))
    recommended_students_location.sort(key=lambda x : x[1],reverse=True)    
    return recommended_students_location
         
        


### jobsize - companysize

In [20]:
lemma = WordNetLemmatizer()
def lemmatize_skills(skills_str):
    skills = [skill.strip() for skill in skills_str.split(' ')]
    lemmatized_skills = []
    for skill in skills:
        tokens = word_tokenize(skill.lower())
        lemmas = [lemma.lemmatize(token) for token in tokens] 
        lemmatized_skills.append(" ".join(lemmas))
    return " ".join(lemmatized_skills)

students['lemmatized_jobsize'] = students['preferred_job_size'].apply(lemmatize_skills)
company['lemmatized_companysize'] = company['company_size'].apply(lemmatize_skills)

tfi = TfidfVectorizer()
students_jobsize_vectors = tfi.fit_transform(students['lemmatized_jobsize'])
company_companysize_vectors = tfi.transform(company['lemmatized_companysize'])

similarity_score_jobsize = cosine_similarity(students_jobsize_vectors,company_companysize_vectors)

In [21]:
def recommend_companies_jobsize_companysize(student_name_input):
    
    try:
        student_index = students[students['student_name'].str.lower() == student_name_input.lower()].index[0]
    except IndexError:
        print(f"Student with name '{student_name_input}' not found.")
        exit()

    # student_name = students['student_name'][student_index]
    # print(f"Recommendations for {student_name}:")
    company_indices = similarity_score_jobsize[student_index].argsort()[::-1][:19]
    recommended_companies_jobsize = []
    weight_skill = 0.1
    for company_index in company_indices:
        # print(f"  - {company['com_name'][company_index]}")
        company_name = company['com_name'][company_index]
        similarity = similarity_score_jobsize[student_index, company_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_companies_jobsize.append((company_name,weighted_similarity))
    recommended_companies_jobsize.sort(key=lambda x : x[1],reverse=True)    
    return recommended_companies_jobsize 



In [22]:
def recommend_students_jobsize_companysize(company_name_input):    
    try:
        company_index = company[company['com_name'].str.lower() == company_name_input.lower()].index[0]
    except IndexError:
        print(f"Company with name '{company_name_input}' not found.")
        return 

    # print(f"Recommendations for {company['com_name'][company_index]}:")
    student_indices = similarity_score_jobsize[:, company_index].argsort()[::-1][:10]
    recommended_students_jobsize = []
    weight_skill = 0.1
    for student_index in student_indices:
        # print(f"  - {company['com_name'][company_index]}")
        student_name = students['student_name'][student_index]
        similarity = similarity_score_jobsize[student_index, company_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_students_jobsize.append((student_name,weighted_similarity))
    recommended_students_jobsize.sort(key=lambda x : x[1],reverse=True)    
    return recommended_students_jobsize
         

### category-studentrank

In [23]:
lemma = WordNetLemmatizer()
def lemmatize_skills(skills_str):
    skills = [skill.strip() for skill in skills_str.split(' ')]
    lemmatized_skills = []
    for skill in skills:
        tokens = word_tokenize(skill.lower())
        lemmas = [lemma.lemmatize(token) for token in tokens] 
        lemmatized_skills.append(" ".join(lemmas))
    return " ".join(lemmatized_skills)

students['lemmatized_category'] = students['Category'].apply(lemmatize_skills)
company['lemmatized_student_rank'] = company['student_rank'].apply(lemmatize_skills)

tfi = TfidfVectorizer()
students_category_vectors = tfi.fit_transform(students['lemmatized_category'])
company_studentrank_vectors = tfi.transform(company['lemmatized_student_rank'])

similarity_score_category = cosine_similarity(students_category_vectors,company_studentrank_vectors)

In [24]:
def recommend_companies_rannk_category(student_name_input):
    
    try:
        student_index = students[students['student_name'].str.lower() == student_name_input.lower()].index[0]
    except IndexError:
        print(f"Student with name '{student_name_input}' not found.")
        exit()

    # student_name = students['student_name'][student_index]
    # print(f"Recommendations for {student_name}:")
    company_indices = similarity_score_category[student_index].argsort()[::-1][:19]
    recommended_companies_category_rank = []
    weight_skill = 0.3
    for company_index in company_indices:
        # print(f"  - {company['com_name'][company_index]}")
        company_name = company['com_name'][company_index]
        similarity = similarity_score_category[student_index, company_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_companies_category_rank.append((company_name,weighted_similarity))
    recommended_companies_category_rank.sort(key=lambda x : x[1],reverse=True)    
    return recommended_companies_category_rank 


In [25]:
def recommend_students_rannk_category(company_name_input):
    try:
        company_index = company[company['com_name'].str.lower() == company_name_input.lower()].index[0]
    except IndexError:
        print(f"Company with name '{company_name_input}' not found.")
        return 

    # print(f"Recommendations for {company['com_name'][company_index]}:")
    student_indices = similarity_score_category[:, company_index].argsort()[::-1][:10]
    recommended_companies_category_rank = []
    weight_skill = 0.3
    for student_index in student_indices:
        # print(f"  - {company['com_name'][company_index]}")
        student_name = students['student_name'][student_index]
        similarity = similarity_score_category[student_index, company_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_companies_category_rank.append((student_name,weighted_similarity))
    recommended_companies_category_rank.sort(key=lambda x : x[1],reverse=True)    
    return recommended_companies_category_rank
        


### bottom carousel

In [26]:
#recommending companies to students
def merge_and_aggregate_recommendations(skill_recommendations, role_recommendations, location_recommendations,
                                        jobsize_recommendations, category_recommendations):
    

    combined_scores = {}  
    def add_scores_from_list(recommendations, weight):
        if isinstance(recommendations, str): 
            return
        for company, score in recommendations:
            combined_scores[company] = combined_scores.get(company, 0) + score * weight

   
    add_scores_from_list(skill_recommendations, 0.5) 
    add_scores_from_list(role_recommendations, 0.6)   
    add_scores_from_list(location_recommendations, 0.2) 
    add_scores_from_list(jobsize_recommendations, 0.1) 
    add_scores_from_list(category_recommendations, 0.3)
    sorted_recommendations = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)

    return sorted_recommendations


def recommend_companies_combined(student_name_input):
    skill_recommendations = recommend_companies_skills_prerequisites(student_name_input)
    role_recommendations = recommend_companies_preferredrole_jobtitle(student_name_input) 
    location_recommendations = recommend_companies_location_location(student_name_input)
    jobsize_recommendations = recommend_companies_jobsize_companysize(student_name_input)
    category_recommendations = recommend_companies_rannk_category(student_name_input)
     

    combined_recommendations = merge_and_aggregate_recommendations(
        skill_recommendations, role_recommendations, location_recommendations,
        jobsize_recommendations, category_recommendations
    )

    return combined_recommendations

student_name_input = input("Enter student name: ")
recommendations = recommend_companies_combined(student_name_input)

if recommendations:
    print(f"Combined Recommendations for {student_name_input}:")
    for company, score in recommendations:
        print(f"  - {company} (Combined Score: {score:.4f})")
        
# recommendations

Combined Recommendations for prakarsh singh:
  - Tata Consultancy Services (Combined Score: 0.3665)
  - Adobe (Combined Score: 0.3049)
  - Capgemini (Combined Score: 0.2921)
  - Infosys (Combined Score: 0.2894)
  - Flipkart (Combined Score: 0.2794)
  - upGrad (Combined Score: 0.2783)
  - Google (Combined Score: 0.2643)
  - 1mg (Combined Score: 0.2603)
  - Wipro (Combined Score: 0.2575)
  - Microsoft (Combined Score: 0.1934)
  - Amazon (Combined Score: 0.1800)
  - Paytm (Combined Score: 0.1728)
  - Mindtree (Combined Score: 0.1697)
  - Cognizant (Combined Score: 0.1673)
  - Razorpay (Combined Score: 0.1505)
  - Meesho (Combined Score: 0.1356)
  - Urban Company (Combined Score: 0.1318)
  - Ola (Combined Score: 0.0508)


In [27]:
top_company = recommendations[0]
top_company

('Tata Consultancy Services', 0.3665268984819888)

In [28]:
company = pd.read_excel("companydataset.xlsx")
company.drop(['Unnamed: 0','Unnamed: 6'],axis=1,inplace=True)

In [29]:
#recommending students to comapnies
def merge_and_aggregate_recommendations(skill_recommendations, role_recommendations, location_recommendations,
                                        jobsize_recommendations, category_recommendations):
    

    combined_scores = {}  
    def add_scores_from_list(recommendations, weight):
        if isinstance(recommendations, str): 
            return
        for students, score in recommendations:
            combined_scores[students] = combined_scores.get(students, 0) + score * weight

   
    add_scores_from_list(skill_recommendations, 0.5) 
    add_scores_from_list(role_recommendations, 0.6)   
    add_scores_from_list(location_recommendations, 0.2) 
    add_scores_from_list(jobsize_recommendations, 0.1) 
    add_scores_from_list(category_recommendations, 0.3)
    sorted_recommendations = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)

    return sorted_recommendations


def recommend_companies_combined(company_name_input):
    skill_recommendations = recommend_students_skills(company_name_input)
    role_recommendations = recommend_students_preferredrole_jobtitle(company_name_input) 
    location_recommendations = recommend_students_location(company_name_input)
    jobsize_recommendations = recommend_students_jobsize_companysize(company_name_input)
    category_recommendations = recommend_students_rannk_category(company_name_input)
    # salary_recommendations = recommend_companies_salary(company_name_input) 

    combined_recommendations = merge_and_aggregate_recommendations(
        skill_recommendations, role_recommendations, location_recommendations,
        jobsize_recommendations, category_recommendations
    )

    return combined_recommendations

company_name_input = input("Enter company name: ")
recommendations = recommend_companies_combined(company_name_input)

if recommendations:
    print(f"Combined Recommendations for {company_name_input}:")
    for students, score in recommendations:
        print(f"  - {students} (Combined Score: {score:.4f})")

Combined Recommendations for amazon:
  - Devanshu Thakur (Combined Score: 0.5148)
  - Shubh Gupta (Combined Score: 0.3633)
  - Purvi Wasan (Combined Score: 0.3482)
  - Ayush Kirdak (Combined Score: 0.3271)
  - Arjun Deshpande (Combined Score: 0.2812)
  - Aditya Chunarkar (Combined Score: 0.2561)
  - Prakarsh Singh (Combined Score: 0.1800)
  - Diya Runwal (Combined Score: 0.1445)
  - Bhoomika Gupta (Combined Score: 0.1000)
  - Amitesh Bhattacharya (Combined Score: 0.0900)
  - Varun Patney (Combined Score: 0.0900)
  - Ishaan Choudhary (Combined Score: 0.0900)
  - Ansh Kirdak (Combined Score: 0.0695)
  - Darshan Raj (Combined Score: 0.0639)
  - Chaitanya Muley (Combined Score: 0.0500)
  - Devyanshi Saini (Combined Score: 0.0100)
  - Syed Nousin  (Combined Score: 0.0000)
  - Sakshi Negi (Combined Score: 0.0000)
  - Prisha Sharma (Combined Score: 0.0000)


### swipe left-right function

In [30]:
company = pd.read_excel("companydataset.xlsx")
company.drop(['Unnamed: 0','Unnamed: 6'],axis=1,inplace=True)
company.head()


,com_name,industry,job_title,job_description,prerequisites,location,is_startup,company_size,student_rank,salary
0,Tata Consultancy Services,Technology Services & Consulting,"Software Engineer,Front-End Developer ,Databas...",Join our team to develop scalable software sol...,"cpp,dsa,javascript,sql",Mumbai,0,large,all,600000
1,Google,Technology,"ml Engineer,Software Developer,MERN stack deve...",Build cutting-edge web applications and machin...,"webdev,python,ml,datascience,MERN","Delhi,Banglore,Hyderabad",0,large,advanced,3000000
2,Infosys,Technology Services & Consulting,"Java Developer,Mobile Application Developer,Da...",Develop enterprise-level applications with Jav...,"java,c,flutter,javascript,sql","Bangalore,Hyderabad,Pune",0,large,all,900000
3,Microsoft,Technology,"ml Engineer,Backend Developer,Full stack devel...",Design and implement intelligent systems with ...,"ml,python,MERN,flask,django","Banglore,Hyderabad,Noida",0,large,advanced,3000000
4,Amazon,"E-commerce, Cloud Computing, AI","Software Development Engineer,Data Analyst,ml ...",Pioneer machine learning models that drive inn...,"ml,python,datascience,cpp,dsa","Bangalore,Hyderabad",0,large,advanced,3000000


In [31]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
label = LabelEncoder()
scaler = StandardScaler()

In [32]:
#company company similarity matrix
def lemmatize_skills(skills_str):
    skills = [skill.strip() for skill in skills_str.split(' ')]
    lemmatized_skills = []
    for skill in skills:
        tokens = word_tokenize(skill.lower())
        lemmas = [lemma.lemmatize(token) for token in tokens] 
        lemmatized_skills.append(" ".join(lemmas))
    return " ".join(lemmatized_skills)

for col in company.select_dtypes(include='object').columns:
    company[col] = company[col].str.lower().str.replace(',', '', regex=False)

text_columns = ['job_title', 'job_description', 'prerequisites', 'industry', 'location']
for col in text_columns:
    company[col] = company[col].apply(lemmatize_skills)
    
company['combined_text'] = company['job_title'] + " " + company['job_description'] + " " + company['prerequisites'] + " " + company['industry'] + " " + company['location']

# TF-IDF Vectorization
tfidf = TfidfVectorizer()
text_vectors = tfidf.fit_transform(company['combined_text']).toarray()

# Numerical Preprocessing
scaler = StandardScaler()
numerical_features = scaler.fit_transform(company[['salary']])

# Categorical Preprocessing
label_encoder = LabelEncoder()
company['company_size_encoded'] = label_encoder.fit_transform(company['company_size'])
company['student_rank_encoded'] = label_encoder.fit_transform(company['student_rank'])

categorical_features = company[['is_startup', 'company_size_encoded', 'student_rank_encoded']].values

# Combine Features
import numpy as np
feature_vectors = np.concatenate((text_vectors, numerical_features, categorical_features), axis=1)

# Similarity Calculation
company_similarity_matrix = cosine_similarity(feature_vectors)

# Create DataFrame
company_names = company['com_name'].tolist()
company_similarity_df = pd.DataFrame(company_similarity_matrix, index=company_names, columns=company_names)

company_similarity_df

,tata consultancy services,google,infosys,microsoft,amazon,wipro,adobe,flipkart,paytm,ola,cognizant,capgemini,mindtree,razorpay,meesho,urban company,1mg,upgrad
tata consultancy services,1.000000,-0.505068,0.714772,-0.479515,-0.497163,0.681493,0.542621,0.349026,-0.321230,0.364886,0.764442,0.319460,0.663068,0.509812,0.309702,0.263455,-0.081149,0.201172
google,-0.505068,1.000000,-0.355603,0.844494,0.808613,-0.248983,-0.112049,0.304589,0.608322,0.070000,-0.539990,-0.207011,-0.421998,-0.178230,-0.078810,0.004518,0.162672,-0.226585
infosys,0.714772,-0.355603,1.000000,-0.325196,-0.366937,0.729946,0.538183,0.434449,-0.225918,0.423404,0.697040,0.261882,0.721043,0.517238,0.314614,0.275424,-0.060775,0.137689
microsoft,-0.479515,0.844494,-0.325196,1.000000,0.812744,-0.240127,-0.105009,0.293366,0.677312,0.095404,-0.557465,-0.204344,-0.417563,-0.163165,-0.074766,-0.006305,0.160288,-0.228036
amazon,-0.497163,0.808613,-0.366937,0.812744,1.000000,-0.272041,-0.101395,0.229668,0.623315,0.054170,-0.525197,-0.233802,-0.449404,-0.181509,-0.064507,-0.001652,0.157521,-0.243138
wipro,0.681493,-0.248983,0.729946,-0.240127,-0.272041,1.000000,0.612218,0.550805,-0.168320,0.705484,0.658913,0.414263,0.858913,0.798075,0.635523,0.613008,0.269712,0.411617
adobe,0.542621,-0.112049,0.538183,-0.105009,-0.101395,0.612218,1.000000,0.490709,-0.083416,0.422339,0.495251,0.092165,0.430032,0.478609,0.289596,0.282538,-0.012162,0.047459
flipkart,0.349026,0.304589,0.434449,0.293366,0.229668,0.550805,0.490709,1.000000,0.189258,0.474468,0.273259,-0.009894,0.281320,0.416350,0.322015,0.301271,0.073229,-0.005688
paytm,-0.321230,0.608322,-0.225918,0.677312,0.623315,-0.168320,-0.083416,0.189258,1.000000,0.060687,-0.398249,-0.098172,-0.263679,-0.110042,-0.066343,0.025403,0.124926,-0.171014
ola,0.364886,0.070000,0.423404,0.095404,0.054170,0.705484,0.422339,0.474468,0.060687,1.000000,0.323578,0.470857,0.620444,0.736750,0.674081,0.691262,0.498053,0.463302


In [33]:
def top_carousel_top_company(company_name_input):
    # Ensure the input is a string and clean it up
    if isinstance(company_name_input, tuple):  
        company_name_input = company_name_input[0]  # Extract only the company name

    company_name_input = str(company_name_input).strip().lower() # Ensure it's a string and remove spaces

    
    if company_name_input in company_similarity_df.columns:
        similar_companies = company_similarity_df[company_name_input].sort_values(ascending=False)[1:5]  # Exclude itself
        print(f"Top 5 similar companies for {company_name_input}:")
        for similar_company, similarity_score in similar_companies.items():
            print(f"  - {similar_company}: {similarity_score:.4f}")
        print("-" * 30)
    else:
        print(f"Company '{company_name_input}' not found in the dataset.")



# Ensure only the company name is passed
top_company_name = top_company[0] if isinstance(top_company, tuple) else top_company
top_carousel_top_company(top_company_name)



Top 5 similar companies for tata consultancy services:
  - cognizant: 0.7644
  - infosys: 0.7148
  - wipro: 0.6815
  - mindtree: 0.6631
------------------------------


In [44]:
students = pd.read_excel("studentsdataset.xlsx")
students['skills'] = students['skills'].str.replace(',',' ')
students['preferred_job_type'] = students['preferred_job_type'].str.lower().replace(',',' ')
students['preferred_job_location'] = students['preferred_job_location'].str.lower().replace(',',' ')
students['preferred_job_size'] = students['preferred_job_size'].str.lower().replace(',',' ')
students['preferred_role'] = students['preferred_role'].str.lower().replace(',',' ')
students['Category'] = students['Category'].str.lower()
students.head(1)

,student_id,student_name,year,CGPA,skills,preferred_job_type,preferred_job_location,preferred_job_size,preferred_role,Category,expected_salary_pa,expected_salary_pm
0,232,Shubh Gupta,4,9.69,cpp python dsa ml nlp dl,internship,mumbai,midsize,ml intern,advanced,0,68000.0


In [ ]:

import random



# Simulating User Swipes (1 = Right Swipe, 0 = Left Swipe)
student_swipes = {(1, 101): 1, (2, 102): 1, (3, 103): 0, (4, 104): 1}  # Student -> Recruiter swipes
recruiter_swipes = {(101, 1): 1, (102, 2): 1, (103, 3): 1, (104, 4): 1}  # Recruiter -> Student swipes

# Function to check mutual acceptance and unlock chat
def check_mutual_acceptance():
    matches = []
    for (s_id, r_id), s_swipe in student_swipes.items():
        if s_swipe == 1 and recruiter_swipes.get((r_id, s_id), 0) == 1:
            matches.append((s_id, r_id))
    return matches

# Find matches and unlock chat
mutual_matches = check_mutual_acceptance()
print("Mutual Matches (Chat Unlocked):", mutual_matches)

# Chat System Placeholder
chats = {match: [] for match in mutual_matches}  # Empty chat history

# Chat Enhancements
def send_message(s_id, r_id, sender, message):
    if (s_id, r_id) in chats:
        chats[(s_id, r_id)].append(f"{sender}: {message}")
    else:
        print("Chat not unlocked!")

def get_chat_history(s_id, r_id):
    return chats.get((s_id, r_id), [])

def suggest_icebreaker():
    icebreakers = [
        "What excites you about this job opportunity?",
        "Can you share a project you're most proud of?",
        "What skills do you think are most valuable in this role?",
        "What do you look for in a great team culture?"
    ]
    return random.choice(icebreakers)

# Example Messages
send_message(1, 101, "Alice", "Hi, I'm interested in this role!")
send_message(1, 101, "TechCorp", "Great! Let's discuss further.")
send_message(1, 101, "System", f"Icebreaker: {suggest_icebreaker()}")

# Display Chat History
for match, messages in chats.items():
    print(f"Chat between Student {match[0]} & Recruiter {match[1]}:")
    for msg in messages:
        print(msg)
